In [8]:
from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import pyspark
from pyspark.ml import feature, regression, Pipeline, classification
from pyspark.sql import functions as fn, Row
from pyspark import sql
import re
import time

import matplotlib.pyplot as plt
import pandas as pd

In [19]:
flights_nb_clean = spark.read.parquet('flights_transformed.parquet')
flights_nb_clean = flights_nb_clean.withColumn('label', fn.col('DEPARTURE_DELAY_index')).drop('DEPARTURE_DELAY_index')
flights_nb_clean.show(1)

+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+----------+--------------+--------+---------+-----------------+------------+-------------+--------+-------------+-------------------+-----------------+--------------------+-------------------------+-------------------------+------------------+--------------------+-----------------------+---------------------+---------------------------+-------------------------+----------------------------+---------------------------------+---------------------------------+--------------------------+----------------------------+-------------------------------+--------------------+-----+
|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|WHEELS_OFF|SCHEDULED_TIME|DISTANCE|WHEELS_ON|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|AIRLINE_index|FLIGHT_NUMBER_index|TAIL

In [20]:
# Make a training, validation, and test split #
training_df, validation_df, testing_df = flights_nb_clean.randomSplit([0.6, 0.3, 0.1], seed=0)

In [21]:
nb_model = classification.NaiveBayes(smoothing=1).fit(training_df)

In [22]:
# Transform it against the validation data #
nb_df = nb_model.transform(validation_df).select('prediction','label').toPandas()

In [23]:
# Show the results #
evaluation_nb_df = pd.concat([nb_df['prediction'],nb_df['label']], 
                          axis = 1, keys=['prediction','label'])
crosstab_nb_df = pd.crosstab(evaluation_nb_df.prediction, evaluation_nb_df.label)
display(crosstab_nb_df)
print("Accuracy: {:,.3f}%" .format((crosstab_nb_df[0][0] + crosstab_nb_df[1][1]) / crosstab_nb_df.values.sum() * 100))

label,0.0,1.0
prediction,,
0.0,11768,5016
1.0,4728,4924


Accuracy: 63.141%
